In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dnd_monsters.csv')
df.head()

,name,url,cr,type,size,ac,hp,speed,align,legendary,source,str,dex,con,int,wis,cha
0,aarakocra,https://www.aidedd.org/dnd/monstres.php?vo=aar...,1/4,humanoid (aarakocra),Medium,12,13,fly,neutral good,NaN,Monster Manual (BR),10.0,14.0,10.0,11.0,12.0,11.0
1,abjurer,NaN,9,humanoid (any race),Medium,12,84,NaN,any alignment,NaN,Volo's Guide to Monsters,NaN,NaN,NaN,NaN,NaN,NaN
2,aboleth,https://www.aidedd.org/dnd/monstres.php?vo=abo...,10,aberration,Large,17,135,swim,lawful evil,Legendary,Monster Manual (SRD),21.0,9.0,15.0,18.0,15.0,18.0
3,abominable-yeti,NaN,9,monstrosity,Huge,15,137,NaN,chaotic evil,NaN,Monster Manual,NaN,NaN,NaN,NaN,NaN,NaN
4,acererak,NaN,23,undead,Medium,21,285,NaN,neutral evil,NaN,Adventures (Tomb of Annihilation),NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
'''я убираю все строки, где url пустое (по совпадению все строки без статов)'''
df['legendary'] = df['legendary'].fillna("No")
df.dropna(subset=['url'], inplace=True)
print(f'пропуски в ссылках: {df['url'].isna().sum()}')
print(df.shape)
df.head()

пропуски в ссылках: 0
(401, 17)


,name,url,cr,type,size,ac,hp,speed,align,legendary,source,str,dex,con,int,wis,cha
0,aarakocra,https://www.aidedd.org/dnd/monstres.php?vo=aar...,1/4,humanoid (aarakocra),Medium,12,13,fly,neutral good,No,Monster Manual (BR),10.0,14.0,10.0,11.0,12.0,11.0
2,aboleth,https://www.aidedd.org/dnd/monstres.php?vo=abo...,10,aberration,Large,17,135,swim,lawful evil,Legendary,Monster Manual (SRD),21.0,9.0,15.0,18.0,15.0,18.0
5,acolyte,https://www.aidedd.org/dnd/monstres.php?vo=aco...,1/4,humanoid (any race),Medium,10,9,NaN,any alignment,No,Monster Manual (SRD),10.0,10.0,10.0,10.0,14.0,11.0
6,adult-black-dragon,https://www.aidedd.org/dnd/monstres.php?vo=adu...,14,dragon,Huge,19,195,"fly, swim",chaotic evil,Legendary,Monster Manual (SRD),23.0,14.0,21.0,14.0,13.0,17.0
8,adult-blue-dragon,https://www.aidedd.org/dnd/monstres.php?vo=adu...,16,dragon,Huge,19,225,fly,lawful evil,Legendary,Monster Manual (SRD),25.0,10.0,23.0,16.0,15.0,19.0


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
df = df.drop(columns=['url', 'cr', 'type', 'align', 'source'])
df = pd.concat([df]*3, ignore_index=True)

df['legendary'] = df['legendary'].map({'Legendary': 1, 'No': 0})
df['is_swimming'] = df['speed'].str.contains('s', na=False).astype(int)
df['is_flying'] = df['speed'].str.contains('f', na=False).astype(int)
df = df.drop(columns=['speed'])

size_mapping = {
    'Tiny': 0,
    'Small': 1,
    'Medium': 2,
    'Large': 3,
    'Huge': 4,
    'Gargantuan': 5
}
df['size'] = df['size'].map(size_mapping)

X = df.drop('name', axis=1)
y = df['name']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=104 ,test_size=0.2, shuffle=True)
df.head()

,name,size,ac,hp,legendary,str,dex,con,int,wis,cha,is_swimming,is_flying
0,aarakocra,2,12,13,0,10.0,14.0,10.0,11.0,12.0,11.0,0,1
1,aboleth,3,17,135,1,21.0,9.0,15.0,18.0,15.0,18.0,1,0
2,acolyte,2,10,9,0,10.0,10.0,10.0,10.0,14.0,11.0,0,0
3,adult-black-dragon,4,19,195,1,23.0,14.0,21.0,14.0,13.0,17.0,1,1
4,adult-blue-dragon,4,19,225,1,25.0,10.0,23.0,16.0,15.0,19.0,0,1


In [5]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.2)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy:.4f}')

Epoch 1/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.0030 - loss: 9.5848 - val_accuracy: 0.0000e+00 - val_loss: 5.9906
Epoch 2/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0058 - loss: 5.8488 - val_accuracy: 0.0104 - val_loss: 5.8639
Epoch 3/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0183 - loss: 5.6096 - val_accuracy: 0.0052 - val_loss: 5.7490
Epoch 4/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0135 - loss: 5.3210 - val_accuracy: 0.0052 - val_loss: 5.6742
Epoch 5/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0363 - loss: 5.1086 - val_accuracy: 0.0104 - val_loss: 5.5733
Epoch 6/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0576 - loss: 4.9351 - val_accuracy: 0.0052 - val_loss: 5.4973
Epoch 7/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0922 - loss: 4.5934 - val_accuracy: 0.0052 - val_loss: 5.4422
Epoch 8/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1147 - loss: 4.3415 - val_accuracy: 0.0052 - val_

In [30]:
input_data = np.array([[3, 17, 135, 1, 21.0, 9.0, 15.0, 18.0, 15.0, 18.0, 1, 0]])

predictions = model.predict(input_data)
predicted_probabilities = predictions[0]

top_n = 3
top_n_indices = np.argsort(predicted_probabilities)[-top_n:][::-1]

top_n_probabilities = predicted_probabilities[top_n_indices]
top_n_classes = label_encoder.inverse_transform(top_n_indices)

for i in range(top_n):
    percentage = top_n_probabilities[i] * 100  # Convert to percentage
    print(f'Predicted label: {top_n_classes[i]}, Probability: {percentage:.2f}%')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Predicted label: young-gold-dragon, Probability: 9.03%
Predicted label: yuan-ti-abomination, Probability: 4.86%
Predicted label: aboleth, Probability: 4.17%


In [37]:
import joblib

model.save('model_build1.keras')
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']